In [5]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [4]:
# Loading data and labels
loaded = np.load('smr_sample_S5.npz')
loaded_data = loaded['data_array']
loaded_labels = loaded['label_array']

In [6]:
cv = StratifiedKFold(n_splits=5, shuffle=False)

# CSP MNE 

### Multiclass classifier : 4 classes : L , R , LR , Rest  

In [10]:
np.unique(loaded_labels, return_counts=True)

(array([0, 1, 2, 3]), array([122,  95, 127, 123], dtype=int64))

In [7]:
import pandas as pd
from bbcpy.pipeline import make_pipeline
import bbcpy.functions.helpers as helpers
from bbcpy.functions.base import ImportFunc
from mne.decoding import CSP
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score

In [11]:
# MNE CSP parameters 
n_components=4
reg=0.2 #float | str | None (default None)allow regularization for covariance estimation|If float (between 0 and 1), shrinkage is used/
log=False  # None | bool (default None)
cov_est='concat' #‘concat’ | ‘epoch’ (default ‘concat’)
transform_into='average_power' #‘average_power’ : the average power of each spatial filter| ‘csp_space’ :  the data in CSP space(default ‘average_power’)
norm_trace=True # bool (default False) Normalize class covariance by its trace.
cov_method_params=None #dict | None Parameters to pass to mne.compute_covariance().
rank=None #None | ‘info’ | ‘full’ | dict
component_order='mutual_info' #‘mutual_info’ | ‘alternate’ (default ‘mutual_info’) 


var = ImportFunc(np.var, axis=2)
csp_mne = CSP(n_components=n_components,
          reg=reg,
          log=log,
          cov_est=cov_est,
          transform_into=transform_into,
          norm_trace=norm_trace,
          cov_method_params=cov_method_params,
          rank=rank,
          component_order=component_order)

csp_pipeline = make_pipeline(   csp_mne,
                                var,
                                np.log,
                                LDA()
                            )
    
# Perform cross-validation and store the results
scores = cross_val_score(csp_pipeline, loaded_data, loaded_labels, error_score='raise')

C:\Users\alioo\Desktop\MA\bbcpy_AutoML\bbcpy_env\lib\site-packages\sklearn\utils\metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


Computing rank from data with rank=None
    Using tolerance 57 (2.2e-16 eps * 62 dim * 4.1e+15  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 51 (2.2e-16 eps * 62 dim * 3.7e+15  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 71 (2.2e-16 eps * 62 dim * 5.2e+15  max singular value)
    Estimated rank (mag): 62
    MAG: rank 62 computed from 62 data channels with 0 projectors
Reducing data rank from 62 -> 62
Estimating covariance using SHRINKAGE
Done.
Computing rank from data with rank=None
    Using tolerance 66 (2.2e-16 eps * 62 dim * 4.8e+15  max singular value)
    Estimated rank (

AxisError: axis 2 is out of bounds for array of dimension 2

# CSP bbcpy toolbox 

### Binary Calssifier : L, R

In [12]:
loaded_data = loaded['data_array']
loaded_labels = loaded['label_array']

In [14]:
# Find indices where label is either 0 or 1
indices = np.where((loaded_labels == 0) | (loaded_labels == 1))

In [15]:
# Select only the data corresponding to class 0 and class 1
selected_data = loaded_data[indices]
selected_loaded_labels = loaded_labels[indices]

In [16]:
np.unique(selected_loaded_labels, return_counts=True)

(array([0, 1]), array([122,  95], dtype=int64))

In [18]:
import pandas as pd
from bbcpy.pipeline import make_pipeline
import bbcpy.functions.helpers as helpers
from bbcpy.functions.base import ImportFunc
from bbcpy.functions.spatial import CSP
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [19]:
# Define the values of excllev to investigate
excllev_values = [None, 1.2, 1.8, 2, 4, 8, 10, 20, 100]


csp_bbcpy = CSP(n_cmps=4,
                excllev=1.8,
                scoring=helpers.evscoring_medvar,
                select=helpers.evselect_directorscut)


var = ImportFunc(np.var, axis=2)

# Create the pipeline for this iteration
csp_pipeline = make_pipeline(
                                csp_bbcpy,
                                var,
                                np.log,
                                LDA()
                            )

# Perform cross-validation and store the results
scores = cross_val_score(csp_pipeline, selected_data, selected_loaded_labels, cv=cv, error_score='raise')


C:\Users\alioo\Desktop\MA\bbcpy_AutoML\bbcpy_env\lib\site-packages\sklearn\utils\metaestimators.py:121: FutureWarning: if_delegate_has_method was deprecated in version 1.1 and will be removed in version 1.3. Use available_if instead.
  warnings.warn(


AttributeError: 'numpy.ndarray' object has no attribute 'y'